# 以下内容来自<<深度学习入门 基于pytorch和tensorflow的理论与实现>>    
  红色石头 著

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from mxnet import nd
#mxnet是amazon的深度学习框架，nd数据可以直接加减乘除，nd.dot(X,Y.T)是X与Y的转置作矩阵乘法。

In [ ]:
x = nd.random.normal(0,1,shape=(3,4))
y = nd.ones((3,4))
z = nd.concat(x,y,dim=0)

In [ ]:
#torch就是pytorch的核心库，torch是服务于torchvision的，用来生成图片，视频数据集，一些流行的模型类和预训练模型。
import torch
import torchvision
#torchvision包括datasets,models,transforms,utils四个模块

In [ ]:
#张量可以在GPU版本pytorch上运行，但是nparray只能在CPU版本的pytorch上运行。
x = torch.tensor([[1,2],[3,4]])
y = torch.tensor([[1,1],[1,1]])
x + y #torch.add(x,y) 和 x+y 都可以实现张量加法
x.mul(y)#对应元素相乘
x.mm(y) #矩阵相乘

In [ ]:
#tensor与nparray可以互相转换，也可以共享内存位置，即一个发生改变另一个也变。
a = np.array([[1,1],[1,1]])
b = torch.from_numpy(a)
np.add(a,1,out=a)
#新建的张量默认保存在CPU里，可以自行移动到GPU之中。
b_cuda = b.cuda()
b_cuda


In [ ]:
#pytorch具有自动求导功能，但是需要在设置张量时定义清除
x = torch.ones(1,4,requires_grad=True)
z = torch.sum(x)
#在pytorch中，每个通过函数计算得到的变量都有一个.grad_fn属性，下面要求z对x,y的偏导数。
z.backward()
print(x.grad)
#在设置.requires_grad = True的时候，他们开始形成一个反向图，
#跟踪应用于他们的每个操作，使用所谓的动态计算图(DCG)计算梯度。
#只有设置成浮点型才可以求导。
#https://blog.csdn.net/u011984148/article/details/99670194
#https://blog.csdn.net/qq_27825451/article/details/89393332?utm_medium=distribute.pc_relevant.none-task-blog-2~default~baidujs_baidulandingword~default-0.no_search_link&spm=1001.2101.3001.4242


In [ ]:
import torch.nn as nn
#利用torch.nn可以很方便的定义损失函数，最简单的是nn.MSELoss
class net_name(nn.Module):
#继承父类nn.Module,nn.Module是torch.nn之中一个十分重要的类，包含各层网络定义及forward方法。
    def __init__(self):
        super(net_name,self).__init__()
        self.fc = nn.Linear(2,1)
        #self.fc = nn.Linear表示对于模型的搭建，2是输入维度，1是输出维度。
    def forward(self,x):
        out = self.fc(x)
        return out



In [ ]:
#优化算法库
'''
optimizer = optim.SGD(net.parameters(),lr=0.01)
optimizer.zero_grad() #开始之前一定进行梯度清零
output = net(input)
loss = nn.MSELoss(output,target)
loss.backward()
loos.backward()
optimizer.step() #梯度更新
'''

In [ ]:
#简单线性回归
import torch.optim as optim
import matplotlib.pyplot as plt
x  = torch.unsqueeze(torch.linspace(-1,1,50),dim=1) #对维度进行压缩去掉维度1
y = 3*x + 0.5*torch.randn(x.size())
plt.scatter(x,y)
plt.show()
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression,self).__init__()
        self.fc = nn.Linear(1,1)
    def forward(self,x):
        out = self.fc(x)
        return out

#模型实例化
model = LinearRegression()

#优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(),lr=5e-3)

#模型训练
num_epochs = 1000
for epoch in range(num_epochs):
    out = model(x) #这里为什么直接调用模型就自动前向传播
    loss = criterion(out,y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1)%20 == 0:
        print('Epoch{}/{},loss:{}'.format(epoch,num_epochs,loss))

In [ ]:
model.eval() #启动测试模式
y_hat = model(x)
plt.scatter(x.numpy(),y_hat.detach().numpy(),c='r',label='拟合直线')
plt.show()
#用以下代码查看模型的w0和w1参数
list(model.named_parameters())

In [ ]:
#Tensorflow是一个采用数据流图进行数值计算的开源软件库，其基本原理是基于图运算，将一个计算图拆分成多个子图，然后并行计算，除此以外tensorflow还支持分布式计算。
import tensorflow as tf
c0 = tf.constant(2,name='c0') #0阶张量
c1 = tf.constant([1,2,3],name = 'c1') #一阶张量
#tf中可以使用name参数对张量指定名称，这样做的好处是可以在Tensorboard中查看张量。

In [ ]:
a = tf.constant([[1,2],[3,4]]) #tf.constant是常量，一旦创建之后其值便不能改变。
b = tf.constant([[5,6],[7,8]])
add = tf.add(a,b,name='add')
sub = tf.subtract(a,b,name='sub')
multiply = tf.multiply(a,b,name='multiply') #逐项相乘
matmul = tf.matmul(a,b,name='matmul') #矩阵乘法
#tensorflow是一个采用数据流图进行数值计算的深度学习框架：首先在python之中定义一个用来计算的图（有向图），然后由tensorflow使用这个图，并通过C++进行计算。

In [ ]:
#感知机与神经网络的区别在于感知机是以线性组合为自变量输出是0或1的分段函数，神经网络是线性组合+激活函数。
#为了解决感知机在阶跃处不可导的情况，用sigmoid函数来替代这个阶跃函数。
#交叉熵损失函数常用于分类问题： Loss = [y_real*log(y_predict) + (1-y_real)*log(1-y_predict)] 
'''逻辑回归的python实现'''
import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
np.random.seed(1)
#导入make_moon数据集
X,Y = sklearn.datasets.make_moons(n_samples=200,noise=0.2)
X,Y = X.T,Y.reshape(1,Y.shape[0])
plt.scatter(X[0,Y[0,:]==0],X[1,Y[0,:]==0],c='r',marker='s')
plt.scatter(X[0,Y[0,:]==1],X[1,Y[0,:]==1],c='b',marker='o')
plt.show()
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y
def optimizer(X,Y,num_iterations=200,learning_rate=0.01):
    cost = []
    m = X.shape[1]
    dim = X.shape[0]
    W = np.zeros((dim,1))
    b = 0
    for i in range(num_iterations):
        Z = np.dot(W.T,X) + b
        Y_hat = sigmoid(Z)
        J = -1.0/m * np.sum(Y*np.log(Y_hat)+(1-Y)*np.log(1-Y_hat))
        cost.append(J)
        dW = 1.0 / m *np.dot(X,(Y_hat - Y).T)
        db = 1.0 / m *np.sum(Y_hat-Y)
        W = W - learning_rate * dW
        b = b - learning_rate * db
        if (i+1) % 20 == 0:
            print('Iteration: {},J:{}'.format(i+1,J))
    return W,b,cost
W,b,cost = optimizer(X,Y,num_iterations=1000,learning_rate=0.1)
plt.plot(cost,c='g')
plt.xlabel('iter')
plt.ylabel('Loss')
plt.show()

In [ ]:
'''神经网络的python实现'''
import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
np.random.seed(1)
#导入make_moon数据集
X,Y = sklearn.datasets.make_moons(n_samples=200,noise=0.2)
X,Y = X.T,Y.reshape(1,Y.shape[0])
m = X.shape[1] #样本个数
dim = X.shape[0] #样本维度
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y
def initialize_parameters(n_x,n_h,n_y):
    np.random.seed(0)
    W1 = np.random.randn(n_h,n_x)
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)
    b2 = np.zeros((n_y,1))
    parameters = {
        'W1':W1,
        'W2':W2,
        'b1':b1,
        'b2':b2,
    }
    #parameters初始化程序返回的是一个字典，这样比直接返回很多参数看起来舒服多了
    return parameters
def forward_propagation(X,parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']
    Z1 = np.dot(W1,X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid(Z2)
    cache = {
        'Z1':Z1,
        'A1':A1,
        'Z2':Z2,
        'A2':A2,
    }
    return A2,cache
#交叉熵损失函数
def compute_loss(A2,Y):
    m = Y.shape[1]
    cross_entropy = -(Y*np.log(A2)+(1-Y)*np.log(1-A2))
    cost = 1.0/m * np.sum(cross_entropy)
    return cost
def back_propagation(X,Y,parameters,cache):
    m = X.shape[1]
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']
    Z1 = cache['Z1']
    Z2 = cache['Z2']
    A1 = cache['A1']
    A2 = cache['A2']
    
    dZ2 = A2 - Y
    dW2 = 1.0 / m * np.dot(dZ2,A1.T)
    db2 = 1.0 / m * np.sum(dZ2,axis=1,keepdims=True)
    dZ1 = 1.0 / m * np.dot(W2.T,dZ2) * (1-np.power(A1,2))
    dW1 = 1.0 / m * np.dot(dZ1,X.T)
    db1 = 1.0 / m * np.sum(dZ1,axis=1,keepdims=True)
    
    grads = {
        'dW1':dW1,
        'dW2':dW2,
        'db1':db1,
        'db2':db2,
    }
    return grads
#更新参数
def update_parameters(parameters,grads,learning_rate=0.1):
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']
    dW1 = grads['dW1']
    dW2 = grads['dW2']
    db1 = grads['db1']
    db2 = grads['db2']
    W1 = W1 - learning_rate * dW1
    W2 = W2 - learning_rate * dW2
    b1 = b1 - learning_rate * db1
    b2 = b2 - learning_rate * db2
    parameters = {
        'W1':W1,
        'W2':W2,
        'b1':b1,
        'b2':b2,
    }
    return parameters

#终于造完轮子啦！开始搭建网络！ 这是一个 dim -> 3 -> 1的分类网络，最后一层激活函数是sigmoid。
def nn_model(X,Y,n_h=3,num_iterations=200,learning_rate=0.1):
    n_x = X.shape[0]
    n_y = 1
    parameters = initialize_parameters(n_x,n_h,n_y)
    for i in range(num_iterations):
        A2,cache = forward_propagation(X,parameters)
        cost = compute_loss(A2,Y)
        grads = back_propagation(X,Y,parameters,cache)
        parameters = update_parameters(parameters,grads,learning_rate)
        
        if (i+1) % 20 == 0:
            print('Iter:{},Loss:{}'.format(i+1,cost))
    return parameters
            
parameters = nn_model(X,Y,n_h = 3,num_iterations=500,learning_rate=0.2)

def predict(X,parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']
    Z1 = np.dot(W1,X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid(Z2)
    Y_pred = np.zeros((1,X.shape[1]))
    Y_pred[A2 > 0.5] = 1
    return Y_pred

#绘制
from matplotlib.colors import ListedColormap
x_min,x_max = X[0,:].min() - 0.5, X[0,:].max() + 0.5
y_min,y_max = X[1,:].min() - 0.5, X[1,:].max() + 0.5
step = 0.001
xx,yy = np.meshgrid(np.arange(x_min,x_max,step),np.arange(y_min,y_max,step))
z = predict(np.c_[xx.ravel(),yy.ravel()].T,parameters) # np.c_按行拼接
z = z.reshape(xx.shape)
plt.contourf(xx,yy,z,cmap=plt.cm.Spectral) #绘制边界
plt.scatter(X[0,Y[0,:]==0],X[1,Y[0,:]==0],c='g',marker='s',label='group 1 ')
plt.scatter(X[0,Y[0,:]==1],X[1,Y[0,:]==1],c='y',marker='o',label='group 2 ')
plt.legend()
plt.show()



----------------------------------------优化神经网络----------------------------------------

传统的梯度下降算法在深层网络中表现得并不好。
在训练神经网络时，要尽量避免欠拟合和过拟合。让模型既可以在训练集上准确率较高，同时又有很好的泛化能力。
欠拟合 和 过拟合 分别对应 高偏差 和 高方差。偏差度量了算法的期望预测值与真实结果的偏离程度。
                                      方差度量了训练集的变化导致学习性能的变化。
                                      噪声表示任何学习算法的泛化能力的下界，描述了学习问题本身的难度。
误差 = 偏差 + 方差 + 噪声

一般情况下我们将数据集分成三个部分，训练集用来训练神经网络算法模型，验证集用来验证不同算法的表现情况，以便从中选择最好的算法进行测试。测试集用于测试算法的实际表现，作为该算法的无偏估计。
一般按照60%，20%，20%划分。
一般情况下可以根据训练集和测试集的准确率来判断是否发生了过拟合。
欠拟合时增加网络复杂度。
过拟合时：(1) L1,L2正则化。(2) Dropout. (3) 增加训练样本（数据增强）
一般只对W进行正则化，不对b进行正则化。
L1正则化的最优解边界是正方形，L2正则化是圆形，因此L1正则化更具稀疏性。
L2正则化使模型的解偏向于范数较小的W，L1的优良性能是产生稀疏解，导致W中很多项变成0，计算量小，只保留模型的关键特征。

Dropout正则化的传统方法是：在模型训练阶段,每层的所有神经元都将以概率p保留，在模型测试阶段保留所有神经元，但是每层神经元输出激活值*p，这样做是为了保证训练和测试时输出同期望。
现在更常用的一种正则化方法叫做Inverted Dropout：在模型训练阶段，每层的所有神经元以概率p保留，然后神经元的输出/p。这么做是为了提高模型测试时的运算速度，简化模型。
而且如果改了p值，只需要修改训练阶段的代码。
使用Dropout时候有以下几点实用性建议：
(1)不同隐藏层的keep_prob可以不同。一般来说神经元较多的隐藏层keep_prob可以小一点，反之可以多一点。
(2)不建议对输入层dropout.
(3)越容易过拟合的隐藏层，kepp_prob越小，可以采用交叉验证的测试。

数据增强(图片)
color jittering:图像亮度，饱和度，对比度
random scale:尺度变换
random crop:随机裁剪缩放
horizontal/vertical flip：水平/垂直翻转
shift:平移变换
rotation/reflection:旋转/仿射变换
noise:高斯噪声/模糊处理

经过标准化之后，本来分布极不平衡的数据，现在呈现了标准的正态分布，不同输入数值的范围变得相似了。
注意！！！测试集要拿训练集的均值方差进行标准化。
b可以初始化为0，但是W千万不能初始化为0.

神经网络需要调节的超参数较多：学习率，网络层数，各隐藏层单元数，mini-batch样本数，动量降低因子
                         Adam的beta1,beta2 L1，L2得正则化系数。
选择超参数的方法目前只有网格化训练。

In [ ]:
#为什么要进行卷积运算呢？
#边缘检测（edge detection）是图像处理中最常用的算法之一，其他目的是检测图片中包含的边缘信息，例如水平边缘，垂直边缘等轮廓信息。
#   垂直边缘检测算子                            水平边缘检测算子
#   [                                           [
#   [1,0,-1],                                   [1,1,1],
#   [1,0,-1],                                   [0,0,0],
#   [1,0,-1],                                   [-1,-1,-1],
#   ]                                           ]
#因此，对于卷积层来说，如果我们想检测图片的各种边缘特征，而不仅限于垂直边缘和水平边缘，那么卷积核的具体数值一般需要通过模型训练得到。
#可以通过padding填充使得卷积之后的图片大小不变，避免图片边缘对输出的贡献减小

#对于RGB图像我们选用3个卷积核作为1组，1组卷积核只能输出一个类灰度图，得到这个类灰度图的方式是 1组中的3个分别对RGB卷积，然后累加在一起。
#也是说，对于一组卷积核(3个)：输入是RGB三通道，输出是三个单通道累加在一起的单通道
#卷积完了之后的多张单通道类灰度图要分别进行激活 g(feature_map + b)
#最浅层的卷积核倾向于学习原图的点，颜色等基础特征。深层次卷积核学习轮廓特征。

#池化层的作用在于减小featuremap尺寸，提高运算速度，减小噪声影响，让各特征更具有鲁棒性。
#最后的全连接层就是将比如 10*10*5的张量拉伸成500，然后全连接。
#为什么卷积层最后要引入全连接呢？ 直观的说卷积层提取的是局部视野，如果类比眼睛，卷积层就是将图像翻译成神经信号的工具，而全连接层好比我们的大脑，特能利用卷积层提取的特征做出相应的决策。
#CNN的强大之处就在于卷积层强大的特征提取能力，我们完全可以使用卷积神经网络将特征提取出来，然后实用全连接或者决策树，SVM等各种机器学习算法模型来进行分类。
#一般来说前期每张纸大，但是纸张数目少，后面纸面小，但是纸张数目多。
#
#使用LeNet-5识别mnist，准确率达到99.2%，是一款简单而且功能强大的网络。
#Alexnet比LeNet-5更复杂，但性能更强大。

In [ ]:
'''
使用pytorch书写简单的CNN
'''
from torch.utils.data import DataLoader, Dataset
from urllib.request import urlopen
import torch
import torchvision
import torchvision.transforms as transforms
#https://blog.csdn.net/qq_38156104/article/details/107808990
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
transform = transforms.Compose([transforms.ToTensor()]) 
#torchvision.transforms.ToTensor ：把一个取值范围是[0,255]的PIL.Image或者shape为(H,W,C)的numpy.ndarray，
#转换成形状为[C,H,W]，取值范围是[0,1.0]的torch.FloadTensor

In [ ]:
#train_na = np.array(pd.read_csv('../input/mnist-in-csv/mnist_train.csv'))
#test_na = np.array(pd.read_csv('../input/mnist-in-csv/mnist_test.csv'))
#train_na = [train_na[:,1:786].reshape(60000,28,28),train_na[:,0]]
#test_na = [test_na[:,1:786].reshape(10000,28,28),test_na[:,0]]
#im = train_na[0][0,:,:]
#plt.imshow(im, cmap='gray')

In [ ]:
#从CSV数据到dataloader
class MNIST_data(Dataset):    
    def __init__(self, file_path, transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])):
        df = pd.read_csv(file_path)
    
        self.X = df.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
        self.y = torch.from_numpy(df.iloc[:,0].values)    
        self.transform = transform
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])
        
batch_size = 4
train_dataset = MNIST_data('../input/mnist-in-csv/mnist_train.csv')
test_dataset = MNIST_data('../input/mnist-in-csv/mnist_test.csv')
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=False)


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5) # 输入通道：1    输出通道：6     卷积核： 5*5
        self.pool1 = nn.MaxPool2d(2,2) # 2*2的池化核
        self.conv2 = nn.Conv2d(6,16,5)
        self.pool2 = nn.MaxPool2d(2,2) # 2*2的池化核
        self.fc1   = nn.Linear(16*4*4,120)
        self.fc2   = nn.Linear(120,84)
        self.fc3   = nn.Linear(84,10)
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,16*4*4) #拉伸成一维
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
#print(net) 继承自nn.Module的网络可以直接打印查看网络结构。
#回归问题常用均方差作为损失函数nn.MSELoss()，分类问题常用交叉熵作为损失函数nn.CrossEntropyLoss()。
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(net.parameters(),lr=0.0001)


if torch.cuda.is_available():
     net = net.cuda()
     criterion = criterion.cuda()

net.train() 
num_epoches = 5
cost = []
transform = transforms.Compose([transforms.ToTensor()]) 
for epoch in range(num_epoches):
    running_loss = 0.0
    for i,data in enumerate(train_loader,0):
        inputs,labels = data
        inputs,labels = inputs.cuda(),labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i+1) % 2000 == 0:
            print('epoch:{},mini-batch:{},loss:{}'.format(epoch+1,i+1,running_loss/2000))
            cost.append(running_loss/2000)
            running_loss = 0.0

# 使用GPU时一定要加上：
if torch.cuda.is_available():
      net = net.cuda()
      criterion = criterion.cuda()
      
 ......
 
 
 inputs,labels = data
 inputs,labels = inputs.cuda(),labels.cuda()

In [ ]:
#分别验证在train_set和test_set上的准确率。
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for data in train_loader:
        inputs,labels = data
        inputs,labels = inputs.cuda(),labels.cuda()
        outputs = net(inputs)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Acc of trainset = {}'.format(100*correct/total))
    correct = 0
    total = 0
    for data in test_loader:
        inputs,labels = data
        inputs,labels = inputs.cuda(),labels.cuda()
        outputs = net(inputs)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Acc of testset = {}'.format(100*correct/total))  

In [ ]:
#list(net.named_parameters())  #打印参数

传统的前馈神经网络无法解决 样本维度不一致、label维度不一致 的问题。
传统的前馈神经网络不具备共享特征的能力，例如“张三”在句首中是人名，那么在其他地方也可能是人名。这是共享特征的结果。
比如输入“I am hungry”就是分别对每个单词进行onehot编码，即给每个单词找到一个对应序号。生成的向量在序号处为1，其余处为0，向量长度与单词表的长度相同。
RNN的输出序列y与输入序列x大小相等。
a是记忆单元，是当前层的输出也是下一层的输入。这样就能实现将序列信号中单个信号的信息传递给后面的信号，实现信号的记忆和传递功能。
a<0>一般是零向量。
有时候可能距离很远的两个单词具有很强的联系性，但是由于梯度消失无法在网络中表达出来。
因此我们使用了GRU（Gated Recurrent Unit）解决这一问题。

In [ ]:
'''
使用RNN识别MNIST数据集
'''
#先执行前面的导入torch库，并定义train_loader,test_loader
class RNNnet(nn.Module):
    def __init__(self):
        super(RNNnet,self).__init__()
        self.rnn = nn.LSTM(input_size=28,hidden_size=84,num_layers=2,batch_first=True)
        self.out = nn.Linear(84,10)
    def forward(self,x):
        r_out,(h_n,h_c) = self.rnn(x,None)
        #选择图片最后一行作为RNN模型输出
        out = self.out(r_out[:,-1,:])
        return out
net = RNNnet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.0001)
net = net.cuda()
criterion = criterion.cuda()
#optimizer没有cuda方法嗷！
num_epoches = 5
cost = []
for epoch in range(num_epoches):
    running_loss = 0.0
    for i,data in enumerate(train_loader,0):
        inputs,labels = data
        inputs,labels = inputs.cuda(),labels.cuda()
        inputs = inputs.view(-1,28,28)
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:
            print("epoch:{},mini-batch:{},loss:{}".format(epoch+1,i+1,running_loss/2000))
            cost.append(running_loss/2000) 
            running_loss = 0.0
    
        